In [1]:
import pandas as pd

In [2]:
df=pd.read_excel('Final_Dataset.xlsx')

In [ ]:
df.head(5)

In [110]:
df['Helpfulness_Label']

0         0
1         1
2         0
3         0
4         0
         ..
247710    0
247711    1
247712    1
247713    0
247714    0
Name: Helpfulness_Label, Length: 247715, dtype: int64

In [5]:
import warnings;
warnings.filterwarnings('ignore')


In [18]:
from gensim.models import Word2Vec 
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [4]:
df['Review_Body']=df['Review_Body'].astype(str)

In [4]:
import re
cleaned_review_body=df['Review_Body'].str.replace('[^a-zA-Z]'," ")

In [5]:
cleaned_review=cleaned_review_body.str.split()


In [6]:
cleaned_review

0         [love, these, brushes, and, very, soft, founda...
1         [I, don, t, suffer, from, acne, However, I, su...
2                                            [Great, socks]
3         [I, got, absolutely, no, instructions, with, m...
4         [These, pliers, are, a, great, find, for, fish...
                                ...                        
247710                              [Product, as, promised]
247711    [Locks, securely, but, the, combo, is, horribl...
247712    [i, totally, LOVE, it, at, last, i, can, enjoy...
247713    [Amazing, smell, Arrived, before, ETA, Really,...
247714    [Fits, over, mat, perfectly, No, slipping, on,...
Name: Review_Body, Length: 247715, dtype: object

In [7]:
import nltk 

In [29]:
nltk.download("punkt")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ameli\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [9]:
cleaned_review=cleaned_review.apply(lambda x: [word for word in x if word not in stop_words])

In [142]:
#from nltk.stem import SnowballStemmer

In [143]:
#stemmer= SnowballStemmer('english')

In [144]:
#cleaned_review_body=cleaned_review_body.apply(lambda x: [stemmer.stem(word) for word in x ])

In [10]:
cleaned_review=cleaned_review.apply(lambda x: [word for word in x if len(word)>3 and len(word)<12])

In [75]:
#cleaned_review_body.apply(lambda x: [word.replace('\i$', 'y') for word in x]) 

0         [love, these, brush, veri, soft, foundat, brus...
1         [suffer, from, howev, suffer, from, compuls, s...
2                                             [great, sock]
3         [absolut, instruct, with, vinyl, assum, heat, ...
4         [these, plier, great, find, fishermen, they, e...
                                ...                        
247710                                    [product, promis]
247711    [lock, secur, combo, horribl, bought, this, mo...
247712    [total, love, last, enjoy, coffe, worri, about...
247713    [amaz, smell, arriv, befor, realli, satisfi, p...
247714    [over, perfect, slip, your, sweat, great, prod...
Name: Review_Body, Length: 247715, dtype: object

In [119]:
#model = Word2Vec(cleaned_review_body, min_count=2,size= 100,workers=3, window =3, sg = 0)

In [4]:
import gensim.downloader as api

In [5]:
model_1 = api.load("glove-twitter-100")

In [56]:
model_1.vector_size

100

In [161]:
reviews=[sen for sen in df['Review_Body']]

In [163]:
reviews=" ".join(reviews)

In [32]:
from nltk.tokenize import word_tokenize

cleaned_review=df['Review_Body'].apply(word_tokenize)

In [12]:
cleaned_review=cleaned_review.apply(lambda x: [word.lower() for word in x])

In [35]:
cleaned_review=cleaned_review.apply(lambda x:[word for word in x if len(word)>3 and len(word)<12])

In [40]:
cleaned_review=cleaned_review.apply(lambda x:[word for word in x if word not in
stop_words])

In [175]:
vector_list = [model_1[word] for word in tokens if word in model_1.wv.vocab]

In [176]:
words_list = [word for word in tokens if word in model_1.vocab]


In [217]:
word_vec_zip = zip(words_list, vector_list)

In [218]:
word_vec_dict = dict(word_vec_zip)

In [220]:
df_vectorized_w2vec = pd.DataFrame.from_dict(word_vec_dict, orient='index')

In [227]:
df_vectorized_w2vec.to_csv('vectoriezed_text.csv')

In [3]:
df_vectorized_w2vec=pd.read_csv("vectoriezed_text.csv")

In [10]:
df_vectorized_w2vec=df_vectorized_w2vec.iloc[:,1:]

In [11]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components = 2, init = 'random', random_state = 10, perplexity = 100)

In [12]:
tsne_df = tsne.fit_transform(df_vectorized_w2vec)

In [19]:
tsne_df=pd.DataFrame(tsne_df)
tsne_df.to_csv("tnse_df.csv")

In [8]:
def document_vector(word2vec_model, doc):
    
    doc = [word for word in doc if word in model_1.vocab]
    return np.mean(model_1[doc], axis=0)

In [45]:
def preprocess(text):
    text = text.lower()
    doc = word_tokenize(text)
    doc = [word for word in doc if word not in stop_words]
    doc = [word for word in doc if word.isalpha()] 
    #doc = [word for word in doc if len(word)>4 and len(word)<12]
    return doc

In [10]:
def has_vector_representation(word2vec_model, doc):
    """check if at least one word of the document is in the
    word2vec dictionary"""
    return not all(word not in word2vec_model.vocab for word in doc)

In [11]:
def filter_docs(corpus, texts, condition_on_doc):
    
    number_of_docs = len(corpus)

    if texts is not None:
        texts = [text for (text, doc) in zip(texts, corpus)
                 if condition_on_doc(doc)]

    corpus = [doc for doc in corpus if condition_on_doc(doc)]

    print("{} docs removed".format(number_of_docs - len(corpus)))

    return (corpus, texts,index)

In [48]:
review_list_1=[doc for doc in df['Review_Body']]

In [66]:
review_list=[doc for doc in df['Review_Body']]

In [49]:
cleaned_review_body_1=[preprocess(i) for i in review_list_1]

In [104]:
def process_text(text):
    #text = text.lower()
    doc = word_tokenize(text)
    doc = [word for word in doc if word not in stop_words]
    doc = [word.lower() for word in doc]
    doc = [word for word in doc if word.isalpha()] 
    doc = [word for word in doc if len(word)>3 and len(word)<12]
    return doc

In [105]:
cleaned_review=[process_text(i) for i in df['Review_Body']]

In [ ]:
cleaned_review_body_2=[preprocess(i) for i in review_list_1]

In [51]:
cleaned_review_body_1, review_list_1 = filter_docs(cleaned_review_body_1, review_list_1, lambda doc: has_vector_representation(model_1, doc))

654 docs removed


In [68]:
len(review_list)

247715

In [69]:
cleaned_review_body_1, review_list_1 = filter_docs(cleaned_review_body_1, review_list_1, lambda doc: (len(doc) != 0))

0 docs removed


In [52]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [53]:
detokenized_1=[TreebankWordDetokenizer().detokenize(word) for word in cleaned_review_body_1]

In [ ]:
detokenized_2=[TreebankWordDetokenizer().detokenize(word) for word in cleaned_review_body_2]

In [55]:
detokenized_removed=pd.DataFrame(detokenized_1)

In [ ]:
detokenized_original=pd.DataFrame(detokenized_2)

In [60]:
detokenized_removed.shape

(247061, 1)

In [61]:
import numpy as np
x = []
for doc in cleaned_review_body_1: 
    x.append(document_vector(model_1, doc))
    
X = np.array(x)

In [81]:
pd.DataFrame(X)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.335593,-0.183956,0.084420,0.182747,0.095725,0.084332,0.139901,-0.098358,0.049651,-0.034888,...,-0.092017,0.050891,0.239327,-0.400130,-0.044502,-0.609984,-0.123225,-0.118359,-0.098331,0.388853
1,0.057332,-0.145008,-0.077507,0.099913,-0.001726,0.239505,0.195212,-0.205908,-0.074108,-0.073762,...,-0.098827,0.053384,0.253159,0.074217,0.025746,-0.202337,-0.085835,0.039945,0.114929,0.032494
2,0.240265,-0.449910,0.255430,0.216341,0.294065,0.479560,0.316960,-0.016728,-0.342130,-0.328275,...,0.025500,0.251695,0.200127,-0.170105,-0.555197,-0.660350,0.013435,0.210020,-0.193558,-0.017185
3,0.102460,0.034363,-0.035178,-0.003887,0.102370,0.165402,0.198205,-0.185169,0.080159,-0.133081,...,-0.155108,0.207881,0.054062,-0.164022,0.103973,-0.031730,-0.151994,0.021097,0.048025,0.083719
4,0.122722,-0.046204,0.000785,-0.031691,-0.045783,-0.017736,0.235787,-0.154965,0.019299,-0.093536,...,0.004821,0.096963,-0.020856,-0.116877,-0.030589,-0.004493,-0.019890,-0.082874,-0.166574,0.228325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247056,0.012715,0.286215,-0.193237,0.259055,-0.175520,0.259013,0.124567,-0.454435,0.060769,-0.079935,...,-0.072250,0.261558,0.579485,-0.309292,0.173675,-0.491285,-0.395736,-0.197971,0.289995,0.171095
247057,0.212731,0.135581,0.079503,-0.025656,0.072885,0.118328,0.260708,-0.080643,0.076851,0.041034,...,-0.156632,0.105361,0.026708,-0.099650,-0.105490,-0.145882,-0.066391,-0.072009,0.037982,-0.032683
247058,-0.169992,0.054092,0.053043,-0.038903,-0.209773,0.197966,0.430321,-0.123014,0.182677,-0.025336,...,0.036232,0.195696,0.111377,-0.016318,-0.054676,-0.187137,-0.116381,-0.025897,-0.117413,-0.224302
247059,0.110455,0.361118,0.070931,-0.050831,0.119876,0.214450,-0.122811,-0.223898,0.089509,-0.222808,...,0.043082,0.552421,0.487870,-0.170385,-0.034806,-0.062689,-0.028584,-0.177308,0.224070,-0.187569


In [79]:
r=pd.DataFrame(review_list)
r.to_csv('r.csv')

In [80]:
r_1=pd.DataFrame(review_list_1)
r_1.to_csv('r_1.csv')

In [9]:
from sklearn.manifold import TSNE

# Initialize t-SNE
tsne = TSNE(n_components = 2, init = 'random', random_state = 10, perplexity = 100)

In [ ]:
tsne_df = tsne.fit_transform(X_df)

In [ ]:
tsne_df=pd.DataFrame(tsne_df)
final_removed=pd.concat([detokenized_removed,tsne_df],axis=1)

In [61]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [62]:
tagged_data = [TaggedDocument(words=_d, tags=[df['Helpfulness_Label'][i]]) for i, _d in enumerate(cleaned_review)]

In [ ]:
tagged_data;

In [64]:
import multiprocessing

In [65]:
from tqdm import tqdm

In [66]:
cores = multiprocessing.cpu_count()

In [67]:
model_dbow = Doc2Vec(dm=1, vector_size=25, negative=5, hs=0, min_count=2, sample = 0, workers=cores, alpha=0.025, min_alpha=0.001)

In [68]:
model_dbow.build_vocab(tagged_data)

In [69]:
model_dbow.train(tagged_data,total_examples=len(tagged_data), epochs=30)

In [70]:
model_dbow.save('d2v_25.model')

In [71]:
m=gensim.models.Doc2Vec.load("d2v_25.model")

In [72]:
start_alpha=0.01
infer_epoch=50
X=[]
for d in cleaned_review:
     
    X.append( m.infer_vector(d, alpha=start_alpha, steps=infer_epoch) )

In [73]:
import numpy as np
np.size(X,1)

25

In [85]:
X_df=pd.DataFrame(X)

In [88]:
X_df.to_csv('X_df.csv')

In [4]:
X_df=pd.read_csv('X_df.csv')

In [7]:
X_df=X_df.drop(['Unnamed: 0'],axis=1)

In [76]:
from sklearn.cluster import KMeans

In [ ]:
kmean = KMeans(n_clusters=25, random_state=0)
kmean_cluster=kmean.fit(X)

In [78]:
df['clusters']=kmean_cluster.labels_

In [82]:
df[['clusters']]

,clusters
0,8
1,20
2,1
3,22
4,10
...,...
247710,5
247711,13
247712,2
247713,24


In [ ]:
for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

In [ ]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors 

In [15]:
from sklearn.decomposition import LatentDirichletAllocation as LDA
number_topics = 5
number_words = 10

lda = LDA(n_components=number_topics, n_jobs=-1)


In [17]:
cleaned_review

0         [love, brushes, soft, foundation, brush, washe...
1         [suffer, acne, however, suffer, compulsive, sk...
2                                            [great, socks]
3         [absolutely, vinyl, assumed, heat, vinyl, coul...
4         [these, pliers, great, find, fishermen, they, ...
                                ...                        
247710                                  [product, promised]
247711    [locks, securely, combo, horrible, bought, mou...
247712    [totally, love, last, enjoy, coffee, worry, an...
247713    [amazing, smell, arrived, really, satisfied, p...
247714    [fits, perfectly, slipping, sweat, great, prod...
Name: Review_Body, Length: 247715, dtype: object

In [19]:
dictionary = gensim.corpora.Dictionary(cleaned_review)

In [20]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 always
1 brush
2 brushes
3 cosmetic
4 eyeliner
5 eyeshadow
6 foundation
7 good
8 harsh
9 little
10 love


In [21]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [22]:
bow_corpus = [dictionary.doc2bow(doc) for doc in cleaned_review]

In [23]:

lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [25]:
lda_model[bow_corpus]

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
LDA_text = LatentDirichletAllocation(n_components=5, random_state=42)



In [27]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(lowercase=True, preprocessor = lambda x: x, tokenizer = lambda
x: x)

In [3]:
#topic_vectorized=vectorizer.fit_transform(cleaned_review).toarray()